In [7]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

df_train = pd.read_csv(r"C:\Users\QQ\Desktop\training data\titanic_train.csv")
df_test = pd.read_csv(r"C:\Users\QQ\Desktop\training data\titanic_test.csv")

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)#橫向運算但刪掉有p,s的資料
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()
print(df.shape)


(1309, 10)


In [8]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [15]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]#一維的資料個數
print(train_num)
df.head()


891


,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500


In [22]:
#1 空值補 -1, 做羅吉斯迴歸
df_m1 = df.fillna(-1)
train_X = df_m1[:train_num]
estimator = LogisticRegression()#這行必加,下行才會work
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6981796497395015

In [23]:
# 空值補 0, 做羅吉斯迴歸
df_0 = df.fillna(0)
train_X = df_0[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()


0.6992969681752557

In [29]:
# 空值補平均值, 做羅吉斯迴歸
s=np.mean(df)
df_mean=df.fillna(s)
train_X = df_mean[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6947900320130562

In [32]:
# 空值補中位數, 做羅吉斯迴歸
df_md = df.fillna(df.median())
train_X = df_md[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6947837549431924

In [45]:
#2使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，*搭配羅吉斯迴歸模型*，何者效果最好?
#空值補mean搭配最大最小化，做羅吉斯迴歸
# def normalize_value(x):
#     return 2*((x-np.min(x))/(np.max(x)-np.min(x))-0.5)
# df_1 = normalize_value(df)
# train_X = df_1[:train_num]
# estimator = LogisticRegression()
# cross_val_score(estimator, train_X, train_Y, cv=5).mean()# Input contains NaN
# 空值補平均值, 搭配最大最小化
df = df.fillna(df.mean())
df_temp = MinMaxScaler().fit_transform(df)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()



0.7004142866110099

In [46]:
#2-1空值補0搭配標準化, 做羅吉斯迴歸
df_0 = df.fillna(0)
df_temp = StandardScaler().fit_transform(df_0)
train_X = df_temp[:train_num]
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6959136275186741